In [1]:
import torch
import torch.fft
import numpy as np

# --- 1. CONFIGURATION ---
# Check for Mac GPU (MPS) first, then CUDA, then CPU
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DTYPE= torch.float32 if DEVICE.type == 'mps' else torch.float64


print(f"Simulating on: {DEVICE} ")

# TRUE PARAMETERS
init_sigmasq   = 13.059
init_range_lon = 0.195 
init_range_lat = 0.154 
init_advec_lat = 0.0218
init_range_time = 1.0
init_advec_lon = -0.1689
init_nugget    = 0.247

# Map parameters
init_phi2 = 1.0 / init_range_lon
init_phi1 = init_sigmasq * init_phi2
init_phi3 = (init_range_lon / init_range_lat)**2
init_phi4 = (init_range_lon / init_range_time)**2

# Create Initial Parameters
initial_vals = [np.log(init_phi1), np.log(init_phi2), np.log(init_phi3), 
                np.log(init_phi4), init_advec_lat, init_advec_lon, np.log(init_nugget)]

params_list = [
    # Changed: uses dynamic DTYPE
    torch.tensor([val], requires_grad=True, dtype=torch.float64, device=DEVICE)
    for val in initial_vals
]

# Mean Ozone
OZONE_MEAN = 260.0

# --- 2. EXACT COVARIANCE ---
def get_model_covariance_on_grid(lags_x, lags_y, lags_t, params):
    phi1, phi2, phi3, phi4 = torch.exp(params[0]), torch.exp(params[1]), torch.exp(params[2]), torch.exp(params[3])
    advec_lat, advec_lon = params[4], params[5]
    sigmasq = phi1 / phi2

    u_lat_eff = lags_x - advec_lat * lags_t
    u_lon_eff = lags_y - advec_lon * lags_t
    
    dist_sq = (u_lat_eff.pow(2) * phi3) + (u_lon_eff.pow(2)) + (lags_t.pow(2) * phi4)
    distance = torch.sqrt(dist_sq + 1e-8)
    
    return sigmasq * torch.exp(-distance * phi2)

# --- 3. FFT SIMULATION ---
def generate_exact_gems_field(lat_coords, lon_coords, t_steps, params):
    Nx = len(lat_coords)
    Ny = len(lon_coords)
    Nt = t_steps
    
    print(f"Exact Grid Size: {Nx} (Lat) x {Ny} (Lon) x {Nt} (Time) = {Nx*Ny*Nt} points")
    
    # 1. Calculate Steps
    dlat = float(lat_coords[1] - lat_coords[0])
    dlon = float(lon_coords[1] - lon_coords[0])
    dt = 1.0 
    
    # 2. Padding (2x for non-circular simulation)
    Px, Py, Pt = 2*Nx, 2*Ny, 2*Nt
    
    # 3. Lags Construction
    # Changed: uses dynamic DTYPE
    Lx_len = Px * dlat   
    lags_x = torch.arange(Px, device=DEVICE, dtype=DTYPE) * dlat
    lags_x[Px//2:] -= Lx_len 
    
    Ly_len = Py * dlon   
    lags_y = torch.arange(Py, device=DEVICE, dtype=DTYPE) * dlon
    lags_y[Py//2:] -= Ly_len

    Lt_len = Pt * dt     
    lags_t = torch.arange(Pt, device=DEVICE, dtype=DTYPE) * dt
    lags_t[Pt//2:] -= Lt_len

    # Meshgrid & Covariance
    L_x, L_y, L_t = torch.meshgrid(lags_x, lags_y, lags_t, indexing='ij')
    C_vals = get_model_covariance_on_grid(L_x, L_y, L_t, params)

    # FFT & Convolution
    S = torch.fft.fftn(C_vals)
    S.real = torch.clamp(S.real, min=0)

    # Changed: uses dynamic DTYPE
    random_phase = torch.fft.fftn(torch.randn(Px, Py, Pt, device=DEVICE, dtype=DTYPE))
    weighted_freq = torch.sqrt(S.real) * random_phase
    field_sim = torch.fft.ifftn(weighted_freq).real
    
    return field_sim[:Nx, :Ny, :Nt]

# --- 4. EXECUTION ---

if __name__ == "__main__":
    # Grid: Lat (0 to 5), Lon (123 to 133)
    # Changed: uses dynamic DTYPE
    lats_sim = torch.arange(0, 5.0 + 0.001, 0.044, device=DEVICE, dtype=DTYPE)
    lons_sim = torch.arange(123.0, 133.0 + 0.001, 0.063, device=DEVICE, dtype=DTYPE)
    t_def = 8
    
    print("1. Generating True Field...")
    sim_field = generate_exact_gems_field(lats_sim, lons_sim, t_def, params_list)
    
    print("2. Formatting Output...")
    
    input_map = {}
    aggregated_list = [] 
    
    nugget_std = torch.sqrt(torch.exp(params_list[6]))
    
    # Flip to Descending Order
    lats_flip = torch.flip(lats_sim, dims=[0])
    lons_flip = torch.flip(lons_sim, dims=[0])
    
    grid_lat, grid_lon = torch.meshgrid(lats_flip, lons_flip, indexing='ij')
    flat_lats = grid_lat.flatten()
    flat_lons = grid_lon.flatten()
    
    for t in range(t_def):
        # Flip field to match coordinates
        field_t = sim_field[:, :, t] 
        field_t_flipped = torch.flip(field_t, dims=[0, 1]) 
        
        flat_vals = field_t_flipped.flatten()
        
        # Add Noise + Mean
        obs_vals = flat_vals + (torch.randn_like(flat_vals) * nugget_std) + OZONE_MEAN
        
        time_val = 21.0 + t
        flat_times = torch.full_like(flat_lats, time_val)
        
        row_tensor = torch.stack([flat_lats, flat_lons, obs_vals, flat_times], dim=1)
        
        # Changed: REMOVED .cpu() call. Keeps data on Mac GPU (mps)
        clean_tensor = row_tensor.detach()
        
        key_str = f'2024_07_y24m07day01_hm{t:02d}:53'
        input_map[key_str] = clean_tensor
        aggregated_list.append(clean_tensor)

    aggregated_data = torch.cat(aggregated_list, dim=0)

    print(f"\nDone.")
    print(f"Aggregated Tensor Shape: {aggregated_data.shape}")
    print(f"Device: {aggregated_data.device}")
    print(f"Dtype: {aggregated_data.dtype}")
    
    torch.set_printoptions(precision=4, sci_mode=True)
    print("Sample Output (Lat Desc, Lon Desc):")
    print(aggregated_data[:6])
    
    print(f"\nGradient Check: {aggregated_data.requires_grad} (Should be False)")

Simulating on: cpu 
1. Generating True Field...
Exact Grid Size: 114 (Lat) x 159 (Lon) x 8 (Time) = 145008 points
2. Formatting Output...

Done.
Aggregated Tensor Shape: torch.Size([145008, 4])
Device: cpu
Dtype: torch.float64
Sample Output (Lat Desc, Lon Desc):
tensor([[4.9720e+00, 1.3295e+02, 2.5748e+02, 2.1000e+01],
        [4.9720e+00, 1.3289e+02, 2.6045e+02, 2.1000e+01],
        [4.9720e+00, 1.3283e+02, 2.5836e+02, 2.1000e+01],
        [4.9720e+00, 1.3276e+02, 2.5918e+02, 2.1000e+01],
        [4.9720e+00, 1.3270e+02, 2.5943e+02, 2.1000e+01],
        [4.9720e+00, 1.3264e+02, 2.5507e+02, 2.1000e+01]], dtype=torch.float64)

Gradient Check: False (Should be False)


set up

In [4]:
# Standard libraries
import sys
# Add your custom path
gems_tco_path = "/Users/joonwonlee/Documents/GEMS_TCO-1/src"
sys.path.append(gems_tco_path)
import os
import logging
import argparse # Argument parsing

# Data manipulation and analysis
import pandas as pd
import numpy as np
import pickle
import torch
import torch.optim as optim
import copy                    # clone tensor
import time

# Custom imports


from GEMS_TCO import kernels_reparam_space_time_gpu as kernels_reparam_space_time_gpu


# from GEMS_TCO import kernels_gpu_past1  as kernels_gpu_past1

from GEMS_TCO import orderings as _orderings 
from GEMS_TCO import alg_optimization, alg_opt_Encoder

from typing import Optional, List, Tuple
from pathlib import Path
import typer
import json
from json import JSONEncoder
from GEMS_TCO import configuration as config
from GEMS_TCO.data_loader import load_data2, exact_location_filter
from GEMS_TCO import debiased_whittle
from torch.nn import Parameter

ImportError: cannot import name 'alg_opt_Encoder' from 'GEMS_TCO' (/Users/joonwonlee/Documents/GEMS_TCO-1/src/GEMS_TCO/__init__.py)

In [3]:
from GEMS_TCO import orderings as _orderings

def get_spatial_ordering(
        
        input_maps: torch.Tensor,
        mm_cond_number: int = 10
    ) -> Tuple[np.ndarray, np.ndarray]:
        
        key_list = list(input_maps.keys())
        data_for_coord = input_maps[key_list[0]]
        
        # --- FIX START ---
        # Check if input is Tensor, if so convert to Numpy for KDTree processing
        if isinstance(data_for_coord, torch.Tensor):
            data_for_coord = data_for_coord.cpu().numpy()
        # --- FIX END ---

        x1 = data_for_coord[:, 0]
        y1 = data_for_coord[:, 1]
        
        # Now this works because x1, y1 are numpy arrays
        coords1 = np.stack((x1, y1), axis=-1)

        # Calculate MaxMin ordering
        ord_mm = _orderings.maxmin_cpp(coords1)
        
        # Reorder coordinates to find nearest neighbors
        data_for_coord_reordered = data_for_coord[ord_mm]
        coords1_reordered = np.stack(
            (data_for_coord_reordered[:, 0], data_for_coord_reordered[:, 1]), 
            axis=-1
        )
        
        # Calculate nearest neighbors map
        nns_map = _orderings.find_nns_l2(locs=coords1_reordered, max_nn=mm_cond_number)
        
        return ord_mm, nns_map

ord_mm, nns_map = get_spatial_ordering(input_map, mm_cond_number=10)

In [4]:
mm_input_map = {}
for key in input_map:
    mm_input_map[key] = input_map[key][ord_mm]  # Extract only Lat and Lon columns

# Fit vecchia max min time 2 

In [7]:
import torch
import numpy as np
import time

# --- CONFIGURATION ---
v = 0.5              # Smoothness
mm_cond_number = 8   # Neighbors
nheads = 0           # 0 = Pure Vecchia
lr = 1.0             # LBFGS learning rate
LBFGS_MAX_STEPS = 10
LBFGS_HISTORY_SIZE = 100
LBFGS_LR = 1.0
LBFGS_MAX_EVAL = 100    

#DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

# --- 1. SETUP PARAMETERS (List of Scalars) ---
# Truth: [4.18, 1.94, 0.24, -3.97, 0.014, -0.20, -0.85]
init_sigmasq   = 13.059
init_range_lat = 0.154 
init_range_lon = 0.195
init_advec_lat = 0.0218
init_range_time = 1.0
init_advec_lon = -0.1689
init_nugget    = 0.247

# Map model parameters to the 'phi' reparameterization
init_phi2 = 1.0 / init_range_lon                # 1/range_lon
init_phi1 = init_sigmasq * init_phi2            # sigmasq / range_lon
init_phi3 = (init_range_lon / init_range_lat)**2  # (range_lon / range_lat)^2
init_phi4 = (init_range_lon / init_range_time)**2      # (range_lon / range_time)^2

# Create Initial Parameters (Float64, Requires Grad)
initial_vals = [np.log(init_phi1), np.log(init_phi2), np.log(init_phi3), 
                np.log(init_phi4), init_advec_lat, init_advec_lon, np.log(init_nugget)]

params_list = [
    torch.tensor([val], requires_grad=True, dtype=torch.float64, device=DEVICE)
    for val in initial_vals
]

# --- 2. INSTANTIATE MODEL ---
print(f'\n{"="*40}')
print(f'--- Initializing VecchiaBatched Model ---')
print(f'{"="*40}')

if isinstance(aggregated_data, torch.Tensor):
    aggregated_data = aggregated_data.to(DEVICE)

# Instantiate
model_instance = kernels_reparam_space_time_gpu.fit_vecchia_lbfgs(
    smooth=v,
    input_map=mm_input_map,
    aggregated_data=aggregated_data,
    nns_map=nns_map,
    mm_cond_number=mm_cond_number,
    nheads=nheads
)



# --- 3. OPTIMIZATION LOOP ---
print(f'\n{"="*40}')
print(f'--- Running L-BFGS Optimization ---')
print(f'{"="*40}')

# Optimizer takes the LIST of scalars
optimizer_vecc = model_instance.set_optimizer(
            params_list,     
            lr=LBFGS_LR,            
            max_iter=LBFGS_MAX_EVAL,        
            history_size=LBFGS_HISTORY_SIZE 
        )

start_time = time.time()

out, steps_ran = model_instance.fit_vecc_lbfgs(
        params_list,
        optimizer_vecc,
        # covariance_function argument is GONE
        max_steps=LBFGS_MAX_STEPS, 
        grad_tol=1e-7
    )


end_time = time.time()
epoch_time = end_time - start_time

print(f"\nOptimization finished in {epoch_time:.2f}s.")
print(f"Results after {steps_ran} steps: {out}")
print(f"Final Params: {torch.cat(params_list).detach().cpu().numpy()}")

Using device: cpu

--- Initializing VecchiaBatched Model ---

--- Running L-BFGS Optimization ---
Pre-computing Batched Tensors (Padding Strategy)... Done. Heads: 0, Batched Tails: 145008
--- Starting Batched L-BFGS Optimization (GPU) ---
--- Step 1/10 / Loss: 1.240835 ---
  Param 0: Value=4.2327, Grad=4.93481087913501e-06
  Param 1: Value=1.6988, Grad=-3.534739057027155e-07
  Param 2: Value=0.4977, Grad=1.3392521154350478e-06
  Param 3: Value=-3.6067, Grad=9.785159866847032e-08
  Param 4: Value=0.0256, Grad=2.32525224645036e-06
  Param 5: Value=-0.1789, Grad=1.2585543173616182e-06
  Param 6: Value=-1.9858, Grad=2.4995629127449467e-07
  Max Abs Grad: 4.934811e-06
------------------------------
--- Step 2/10 / Loss: 1.240616 ---
  Param 0: Value=4.2327, Grad=4.93481087913501e-06
  Param 1: Value=1.6988, Grad=-3.534739057027155e-07
  Param 2: Value=0.4977, Grad=1.3392521154350478e-06
  Param 3: Value=-3.6067, Grad=9.785159866847032e-08
  Param 4: Value=0.0256, Grad=2.32525224645036e-06
 

# vecc time 1

In [6]:
import torch
import numpy as np
import time

# --- CONFIGURATION ---
v = 0.5              # Smoothness
mm_cond_number = 12   # Neighbors
nheads = 0           # 0 = Pure Vecchia
lr = 1.0             # LBFGS learning rate
LBFGS_MAX_STEPS = 10
LBFGS_HISTORY_SIZE = 100
LBFGS_LR = 1.0
LBFGS_MAX_EVAL = 100    

#DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

# --- 1. SETUP PARAMETERS (List of Scalars) ---
# Truth: [4.18, 1.94, 0.24, -3.97, 0.014, -0.20, -0.85]
init_sigmasq   = 13.059
init_range_lat = 0.154 
init_range_lon = 0.195
init_advec_lat = 0.0218
init_range_time = 1.0
init_advec_lon = -0.1689
init_nugget    = 0.247

# Map model parameters to the 'phi' reparameterization
init_phi2 = 1.0 / init_range_lon                # 1/range_lon
init_phi1 = init_sigmasq * init_phi2            # sigmasq / range_lon
init_phi3 = (init_range_lon / init_range_lat)**2  # (range_lon / range_lat)^2
init_phi4 = (init_range_lon / init_range_time)**2      # (range_lon / range_time)^2

# Create Initial Parameters (Float64, Requires Grad)
initial_vals = [np.log(init_phi1), np.log(init_phi2), np.log(init_phi3), 
                np.log(init_phi4), init_advec_lat, init_advec_lon, np.log(init_nugget)]

params_list = [
    torch.tensor([val], requires_grad=True, dtype=torch.float64, device=DEVICE)
    for val in initial_vals
]

# --- 2. INSTANTIATE MODEL ---
print(f'\n{"="*40}')
print(f'--- Initializing VecchiaBatched Model ---')
print(f'{"="*40}')

if isinstance(aggregated_data, torch.Tensor):
    aggregated_data = aggregated_data.to(DEVICE)

# Instantiate
model_instance = kernels_reparam_space_time_gpu_past1.fit_vecchia_lbfgs(
    smooth=v,
    input_map=mm_input_map,
    aggregated_data=aggregated_data,
    nns_map=nns_map,
    mm_cond_number=mm_cond_number,
    nheads=nheads
)



# --- 3. OPTIMIZATION LOOP ---
print(f'\n{"="*40}')
print(f'--- Running L-BFGS Optimization ---')
print(f'{"="*40}')

# Optimizer takes the LIST of scalars
optimizer_vecc = model_instance.set_optimizer(
            params_list,     
            lr=LBFGS_LR,            
            max_iter=LBFGS_MAX_EVAL,        
            history_size=LBFGS_HISTORY_SIZE 
        )

start_time = time.time()

out, steps_ran = model_instance.fit_vecc_lbfgs(
        params_list,
        optimizer_vecc,
        # covariance_function argument is GONE
        max_steps=LBFGS_MAX_STEPS, 
        grad_tol=1e-7
    )


end_time = time.time()
epoch_time = end_time - start_time

print(f"\nOptimization finished in {epoch_time:.2f}s.")
print(f"Results after {steps_ran} steps: {out}")
print(f"Final Params: {torch.cat(params_list).detach().cpu().numpy()}")

Using device: cpu

--- Initializing VecchiaBatched Model ---

--- Running L-BFGS Optimization ---
Pre-computing Batched Tensors (Padding Strategy)... Done. Heads: 0, Batched Tails: 145008
--- Starting Batched L-BFGS Optimization (GPU) ---
--- Step 1/10 / Loss: 1.239743 ---
  Param 0: Value=4.2256, Grad=-6.959610383200575e-08
  Param 1: Value=1.6973, Grad=-1.068152907998466e-06
  Param 2: Value=0.4941, Grad=-7.162888808699521e-07
  Param 3: Value=-3.7021, Grad=2.215068929605948e-07
  Param 4: Value=0.0269, Grad=-1.0142547031410931e-07
  Param 5: Value=-0.1839, Grad=-2.9175096793038202e-06
  Param 6: Value=-1.8100, Grad=-1.6566882167386242e-07
  Max Abs Grad: 2.917510e-06
------------------------------
--- Step 2/10 / Loss: 1.239444 ---
  Param 0: Value=4.2256, Grad=-6.959610383200575e-08
  Param 1: Value=1.6973, Grad=-1.068152907998466e-06
  Param 2: Value=0.4941, Grad=-7.162888808699521e-07
  Param 3: Value=-3.7021, Grad=2.215068929605948e-07
  Param 4: Value=0.0269, Grad=-1.0142547031

# fit vecc column

In [7]:
import torch
import numpy as np
import time

# --- CONFIGURATION ---
v = 0.5              # Smoothness
mm_cond_number = 14    # Neighbors
nheads = 113*3           # 0 = Pure Vecchia
lr = 1.0             # LBFGS learning rate
LBFGS_MAX_STEPS = 10
LBFGS_HISTORY_SIZE = 100
LBFGS_LR = 1.0
LBFGS_MAX_EVAL = 100    

#DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

# --- 1. SETUP PARAMETERS (List of Scalars) ---
# Truth: [4.18, 1.94, 0.24, -3.97, 0.014, -0.20, -0.85]
init_sigmasq   = 13.059
init_range_lat = 0.154 
init_range_lon = 0.195
init_advec_lat = 0.0218
init_range_time = 1.0
init_advec_lon = -0.1689
init_nugget    = 0.247

# Map model parameters to the 'phi' reparameterization
init_phi2 = 1.0 / init_range_lon                # 1/range_lon
init_phi1 = init_sigmasq * init_phi2            # sigmasq / range_lon
init_phi3 = (init_range_lon / init_range_lat)**2  # (range_lon / range_lat)^2
init_phi4 = (init_range_lon / init_range_time)**2      # (range_lon / range_time)^2

# Create Initial Parameters (Float64, Requires Grad)
initial_vals = [np.log(init_phi1), np.log(init_phi2), np.log(init_phi3), 
                np.log(init_phi4), init_advec_lat, init_advec_lon, np.log(init_nugget)]

params_list = [
    torch.tensor([val], requires_grad=True, dtype=torch.float32, device=DEVICE)
    for val in initial_vals
]

# --- 2. INSTANTIATE MODEL ---
print(f'\n{"="*40}')
print(f'--- Initializing VecchiaBatched Model ---')
print(f'{"="*40}')

if isinstance(aggregated_data, torch.Tensor):
    aggregated_data = aggregated_data.to(DEVICE)

# Instantiate
model_instance = kernels_gpu_st_simulation_column.fit_vecchia_lbfgs(
    smooth=v,
    input_map=input_map,
    aggregated_data=aggregated_data,
    nns_map=nns_map,
    mm_cond_number=mm_cond_number,
    nheads=nheads
)



# --- 3. OPTIMIZATION LOOP ---
print(f'\n{"="*40}')
print(f'--- Running L-BFGS Optimization ---')
print(f'{"="*40}')

# Optimizer takes the LIST of scalars
optimizer_vecc = model_instance.set_optimizer(
            params_list,     
            lr=LBFGS_LR,            
            max_iter=LBFGS_MAX_EVAL,        
            history_size=LBFGS_HISTORY_SIZE 
        )

start_time = time.time()

out, steps_ran = model_instance.fit_vecc_lbfgs(
        params_list,
        optimizer_vecc,
        # covariance_function argument is GONE
        max_steps=LBFGS_MAX_STEPS, 
        grad_tol=1e-7
    )


end_time = time.time()
epoch_time = end_time - start_time

print(f"\nOptimization finished in {epoch_time:.2f}s.")
print(f"Results after {steps_ran} steps: {out}")
print(f"Final Params: {torch.cat(params_list).detach().cpu().numpy()}")

Using device: cpu

--- Initializing VecchiaBatched Model ---

--- Running L-BFGS Optimization ---
Pre-computing Batched Tensors (Optimized Geometric Strategy)... Done. Heads: 2736, Batched Tails: 142272, Neighbors per point: 14
--- Starting Batched L-BFGS Optimization (GPU) ---
--- Step 1/10 / Loss: 1.252622 ---
  Param 0: Value=4.3421, Grad=7.911366992630064e-07
  Param 1: Value=2.2323, Grad=-1.3886892702430487e-06
  Param 2: Value=0.3790, Grad=5.84215740673244e-07
  Param 3: Value=-54.0534, Grad=-2.3439421640243724e-33
  Param 4: Value=1.6350, Grad=-2.921111796894138e-09
  Param 5: Value=0.4555, Grad=8.524303485302198e-10
  Param 6: Value=-14.6408, Grad=7.244628230651529e-10
  Max Abs Grad: 1.388689e-06
------------------------------
--- Step 2/10 / Loss: 1.239938 ---
  Param 0: Value=4.3421, Grad=7.911366992630064e-07
  Param 1: Value=2.2323, Grad=-1.3886892702430487e-06
  Param 2: Value=0.3790, Grad=5.84215740673244e-07
  Param 3: Value=-54.0534, Grad=-2.3439421640243724e-33
  Para

# fit dw

difference data

In [8]:
a = [11.0474, 0.0623, 0.2445, 1.0972, 0.0101, -0.1671, 1.1825]
day = 0 # 0 index
lat_range= [0,5]
lon_range= [123.0, 133.0]
#lat_range= [1,3]
#lon_range= [125, 129.0]

daily_aggregated_tensors_dw = [aggregated_data]
daily_hourly_maps_dw = [input_map]

db = debiased_whittle.debiased_whittle_preprocess(daily_aggregated_tensors_dw, daily_hourly_maps_dw, day_idx=day, params_list=a, lat_range=lat_range, lon_range=lon_range)


subsetted_aggregated_day = db.generate_spatially_filtered_days(0,5,123,133)
print(subsetted_aggregated_day.shape)
N2= subsetted_aggregated_day.shape[0]
print(N2)
subsetted_aggregated_day[:20]

torch.Size([142832, 4])
142832


tensor([[ 0.0000e+00,  1.2300e+02,  3.8945e-01,  2.1000e+01],
        [ 0.0000e+00,  1.2306e+02,  7.9436e+00,  2.1000e+01],
        [ 0.0000e+00,  1.2313e+02, -9.8576e+00,  2.1000e+01],
        [ 0.0000e+00,  1.2319e+02,  6.2259e+00,  2.1000e+01],
        [ 0.0000e+00,  1.2325e+02, -1.3111e+00,  2.1000e+01],
        [ 0.0000e+00,  1.2331e+02,  8.5856e+00,  2.1000e+01],
        [ 0.0000e+00,  1.2338e+02,  5.7867e+00,  2.1000e+01],
        [ 0.0000e+00,  1.2344e+02, -5.0939e+00,  2.1000e+01],
        [ 0.0000e+00,  1.2350e+02,  7.6338e+00,  2.1000e+01],
        [ 0.0000e+00,  1.2357e+02, -2.4320e+00,  2.1000e+01],
        [ 0.0000e+00,  1.2363e+02, -7.3716e-01,  2.1000e+01],
        [ 0.0000e+00,  1.2369e+02, -4.5453e+00,  2.1000e+01],
        [ 0.0000e+00,  1.2376e+02,  4.4043e+00,  2.1000e+01],
        [ 0.0000e+00,  1.2382e+02,  8.4446e-03,  2.1000e+01],
        [ 0.0000e+00,  1.2388e+02,  3.5328e+00,  2.1000e+01],
        [ 0.0000e+00,  1.2394e+02, -2.7751e+00,  2.1000e+01],
        

In [9]:

dwl = debiased_whittle.debiased_whittle_likelihood()
if __name__ == '__main__':
    start_time = time.time()

    # --- Configuration ---
    DAY_TO_RUN = 3 # data is decided above
    TAPERING_FUNC = dwl.cgn_hamming # Use Hamming taper
    NUM_RUNS = 1
    MAX_STEPS = 20 # L-BFGS usually converges in far fewer steps
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {DEVICE}")

    DELTA_LAT, DELTA_LON = 0.044, 0.063 

    LAT_COL, LON_COL = 0, 1
    VAL_COL = 2 # Spatially differenced value
    TIME_COL = 3


    cur_df =subsetted_aggregated_day
    
    if cur_df.numel() == 0 or cur_df.shape[1] <= max(LAT_COL, LON_COL, VAL_COL, TIME_COL):
        print(f"Error: Data for Day {DAY_TO_RUN} is empty or invalid.")
        exit()

    unique_times = torch.unique(cur_df[:, TIME_COL])
    time_slices_list = [cur_df[cur_df[:, TIME_COL] == t_val] for t_val in unique_times]

    # --- 1. Pre-compute J-vector, Taper Grid, and Taper Autocorrelation ---
    print("Pre-computing J-vector (Hamming taper)...")
    
    # --- 💥 REVISED: Renamed 'p' to 'p_time' 💥 ---
    J_vec, n1, n2, p_time, taper_grid = dwl.generate_Jvector_tapered( 
        time_slices_list,
        tapering_func=TAPERING_FUNC, 
        lat_col=LAT_COL, lon_col=LON_COL, val_col=VAL_COL,
        device=DEVICE
    )

    if J_vec is None or J_vec.numel() == 0 or n1 == 0 or n2 == 0 or p_time == 0:
       print(f"Error: J-vector generation failed for Day {DAY_TO_RUN}.")
       exit()
       
    print("Pre-computing sample periodogram...")
    I_sample = dwl.calculate_sample_periodogram_vectorized(J_vec)

    print("Pre-computing Hamming taper autocorrelation...")
    taper_autocorr_grid = dwl.calculate_taper_autocorrelation_fft(taper_grid, n1, n2, DEVICE)

    if torch.isnan(I_sample).any() or torch.isinf(I_sample).any():
        print("Error: NaN/Inf in sample periodogram.")
        exit()
    if torch.isnan(taper_autocorr_grid).any() or torch.isinf(taper_autocorr_grid).any():
        print("Error: NaN/Inf in taper autocorrelation.")
        exit()

    print(f"Data grid: {n1}x{n2}, {p_time} time points. J-vector, Periodogram, Taper Autocorr on {DEVICE}.")
    # --- END REVISION ---

    # --- 2. Optimization Loop ---
    all_final_results = []
    all_final_losses = []

    for i in range(NUM_RUNS):
        print(f"\n{'='*30} Initialization Run {i+1}/{NUM_RUNS} {'='*30}")

        # --- 7-PARAMETER initialization ---
        ''' 
        init_sigmasq   = 15.0
        init_range_lat = 0.66 
        init_range_lon = 0.7 
        init_nugget    = 1.5
        init_beta      = 0.1  # Temporal range ratio
        init_advec_lat = 0.02
        init_advec_lon = -0.08
        '''
        init_sigmasq   = 13.059
        init_range_lat = 0.154 
        init_range_lon = 0.195
        init_advec_lat = 0.0218
        init_range_time = 0.7
        init_advec_lon = -0.1689
        init_nugget    = 0.247

        init_phi2 = 1.0 / init_range_lon
        init_phi1 = init_sigmasq * init_phi2
        init_phi3 = (init_range_lon / init_range_lat)**2
        # Change needed to match the spatial-temporal distance formula:
        init_phi4 = (init_range_lon / init_range_time)**2      # (range_lon / range_time)^2

        initial_params_values = [
            np.log(init_phi1),    # [0] log_phi1
            np.log(init_phi2),    # [1] log_phi2
            np.log(init_phi3),    # [2] log_phi3
            np.log(init_phi4),    # [3] log_phi4
            init_advec_lat,       # [4] advec_lat (NOT log)
            init_advec_lon,       # [5] advec_lon (NOT log)
            np.log(init_nugget)   # [6] log_nugget
        ]
        
        print(f"Starting with FIXED params (raw log-scale): {[round(p, 4) for p in initial_params_values]}")

        params_list = [
            Parameter(torch.tensor([val], dtype=torch.float64))
            for val in initial_params_values
        ]

        # Helper to define the boundary globally for clarity
        NUGGET_LOWER_BOUND = 0.05
        LOG_NUGGET_LOWER_BOUND = np.log(NUGGET_LOWER_BOUND) # Approx -2.9957

        # --- 💥 REVISED: Use L-BFGS Optimizer 💥 ---
        optimizer = torch.optim.LBFGS(
            params_list,
            lr=1.0,           # Initial step length for line search
            max_iter=20,      # Iterations per step
            history_size=100,
            line_search_fn="strong_wolfe", # Often more robust
            tolerance_grad=1e-5
        )
        # --- END REVISION ---

        print(f"Starting optimization run {i+1} on device {DEVICE} (Hamming, 7-param ST kernel, L-BFGS)...")
        
        # --- 💥 REVISED: Call L-BFGS trainer, pass p_time 💥 ---
        nat_params_str, phi_params_str, raw_params_str, loss, steps_run = dwl.run_lbfgs_tapered(
            params_list=params_list,
            optimizer=optimizer,
            I_sample=I_sample,
            n1=n1, n2=n2, p_time=p_time,
            taper_autocorr_grid=taper_autocorr_grid, 
            max_steps=MAX_STEPS,
            device=DEVICE
        )
        # --- END REVISION ---
        
        if loss is not None:
            all_final_results.append((nat_params_str, phi_params_str, raw_params_str))
            all_final_losses.append(loss)
        else:
            all_final_losses.append(float('inf'))

    print(f"\n\n{'='*25} Overall Result from Run {'='*25} {'='*25}")
    
    valid_losses = [l for l in all_final_losses if l is not None and l != float('inf')]

    if not valid_losses:
        print(f"The run failed or resulted in an invalid loss for Day {DAY_TO_RUN}.")
    else:
        best_loss = min(valid_losses)
        best_run_index = all_final_losses.index(best_loss)
        best_results = all_final_results[best_run_index]
        
        print(f"Best Run Loss: {best_loss} (after {steps_run} steps)")
        print(f"Final Parameters (Natural Scale): {best_results[0]}")
        print(f"Final Parameters (Phi Scale)    : {best_results[1]}")
        print(f"Final Parameters (Raw Log Scale): {best_results[2]}")

    end_time = time.time()
    print(f"\nTotal execution time: {end_time - start_time:.2f} seconds")

Using device: cpu
Pre-computing J-vector (Hamming taper)...
Pre-computing sample periodogram...
Pre-computing Hamming taper autocorrelation...
Data grid: 113x158, 8 time points. J-vector, Periodogram, Taper Autocorr on cpu.

============================== Initialization Run 1/1 ==============================
Starting with FIXED params (raw log-scale): [4.2042, 1.6348, 0.4721, -2.5562, 0.0218, -0.1689, -1.3984]
Starting optimization run 1 on device cpu (Hamming, 7-param ST kernel, L-BFGS)...
--- Step 1/20 ---
 Loss: 1.516678 | Max Grad: 2.666972e-05
  Params (Raw Log): log_phi1: 4.1939, log_phi2: 1.6314, log_phi3: 0.4976, log_phi4: -3.2157, advec_lat: 0.0274, advec_lon: -0.1753, log_nugget: -1.4295
--- Step 2/20 ---
 Loss: 1.505549 | Max Grad: 2.666972e-05
  Params (Raw Log): log_phi1: 4.1939, log_phi2: 1.6314, log_phi3: 0.4976, log_phi4: -3.2157, advec_lat: 0.0274, advec_lon: -0.1753, log_nugget: -1.4295
--- Step 3/20 ---
 Loss: 1.505549 | Max Grad: 2.666972e-05
  Params (Raw Log): log